# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.exceptions import ComputeTargetException
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.widgets import RunDetails

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

### Setup workspace and experiment

In [2]:
ws = Workspace.from_config()

# print some information about the workspace
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

# choose a name for experiment
experiment_name = 'heart-failure-experiment'

experiment=Experiment(ws, experiment_name)

Workspace name: quick-starts-ws-162187
Azure region: southcentralus
Subscription id: d7f39349-a66b-446e-aba6-0053c2cf1c11
Resource group: aml-quickstarts-162187


### Create a cluster

Reuse the AutoML cluster or create a new one, if it doesn't exist

In [3]:
cluster_name = "expcluster"

# Use existing cluster, if it exists
try:
    compute_target = ComputeTarget(workspace=ws, name = cluster_name)
    print('Found existing cluster, use it!')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_DS12_v2',
                                                          max_nodes=6, min_nodes=1)
    compute_target = ComputeTarget.create(workspace=ws, name=cluster_name, provisioning_configuration=compute_config)
compute_target.wait_for_completion(show_output=True)

Found existing cluster, use it!
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


### Create Dataset
Reuse the dataset from the AutoML run or upload it, if it doesn't exist.

In [3]:
from train import read_data
dataset=read_data()
df = dataset.to_pandas_dataframe()
df.describe()

Uploading an estimated of 1 files
Uploading ./data/heart.csv
Uploaded ./data/heart.csv, 1 files out of an estimated total of 1
Uploaded 1 files
Validating arguments.
Arguments validated.
Successfully obtained datastore reference and path.
Uploading file to managed-dataset/2836e50e-adee-43bf-b8a7-44fb85bf2a8e/
Successfully uploaded file to datastore.
Creating and registering a new dataset.
Successfully created and registered a new dataset.


Method register_pandas_dataframe: This is an experimental method, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


,Age,Sex,RestingBP,Cholesterol,FastingBS,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease,ChestPainType_ASY,ChestPainType_ATA,ChestPainType_NAP,ChestPainType_TA,RestingECG_LVH,RestingECG_Normal,RestingECG_ST
count,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000
mean,53.510893,0.789760,132.396514,198.799564,0.233115,136.809368,0.404139,0.887364,0.361656,0.553377,0.540305,0.188453,0.221133,0.050109,0.204793,0.601307,0.193900
std,9.432617,0.407701,18.514154,109.384145,0.423046,25.460334,0.490992,1.066570,0.607056,0.497414,0.498645,0.391287,0.415236,0.218289,0.403770,0.489896,0.395567
min,28.000000,0.000000,0.000000,0.000000,0.000000,60.000000,0.000000,-2.600000,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,47.000000,1.000000,120.000000,173.250000,0.000000,120.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,54.000000,1.000000,130.000000,223.000000,0.000000,138.000000,0.000000,0.600000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
75%,60.000000,1.000000,140.000000,267.000000,0.000000,156.000000,1.000000,1.500000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
max,77.000000,1.000000,200.000000,603.000000,1.000000,202.000000,1.000000,6.200000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [54]:
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice, randint

early_termination_policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling({"--n_estimators": randint(10000),
    "--max_depth": choice(10, 100, 1000, 5000),
    "--min_samples_split": randint(50)})

from azureml.core import Environment
from azureml.core import ScriptRunConfig

env = Environment.from_conda_specification(name = 'env', file_path = './envs/conda_environment.yml')

src = ScriptRunConfig(source_directory = "./",
    script = "train.py",
    compute_target = "expcluster",
    environment = env)

hyperdrive_run_config = HyperDriveConfig(run_config=src,
    hyperparameter_sampling=param_sampling,
    policy=early_termination_policy,
    primary_metric_name="Accuracy",
    primary_metric_goal= PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=50,
    max_concurrent_runs=5)

In [ ]:
#TODO: Submit your experiment
hyperdrive_run = experiment.submit(hyperdrive_run_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
RunDetails(hyperdrive_run).show()

In [ ]:
hyperdrive_run.get_children_sorted_by_primary_metric()

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
best_hyperdrive_run = hyperdrive_run.get_best_run_by_primary_metric()
print("best run details: ", best_hyperdrive_run.get_details())
print("best run metrics :", best_hyperdrive_run.get_metrics())

In [59]:
#TODO: Save the best model
for f in best_hyperdrive_run.get_file_names():
    if f.startswith('outputs/hyperdrive_model.onnx'):
        output_file_path = os.path.join('./hyperdrive_model', 'hyperdrive_model.onnx')
        print('Downloading from {} to {} ...'.format(f, output_file_path))
        best_hyperdrive_run.download_file(name=f, output_file_path=output_file_path)
    elif f.startswith('outputs/model'):
        output_file_path = os.path.join('./hyperdrive_model', 'hyperdrive_model.pkl')
        print('Downloading from {} to {} ...'.format(f, output_file_path))
        best_hyperdrive_run.download_file(name=f, output_file_path=output_file_path)

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [4]:
# this cell is how to register the .pkl model
description = "HyperDrive run heart-failure classification model"

from azureml.core import Model
model = Model.register(workspace=ws,
    model_name="hyperdrive_model",
    model_path='./hyperdrive_model/hyperdrive_model.onnx',
    model_framework=Model.Framework.ONNX,
    model_framework_version='1.3',
    description=description)

Registering model hyperdrive_model


In [40]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core import Model, Environment

service_name = 'heart-failure-hyperdrive-service'

env = Environment.from_conda_specification(name="hyperdrive_env", file_path='./hyperdrive_model/hyperdrive_env.yml')
inference_config = InferenceConfig(entry_script='hyperdrive_model/score_onnx_model_version2.py',
                                   environment=env)
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1,
                                                enable_app_insights=True,
                                                auth_enabled=True)

In [35]:
# deploy to local for debugging
from azureml.core.webservice import LocalWebservice
deployment_config = LocalWebservice.deploy_configuration(port=6789)
test_service = Model.deploy(
    ws,
    name='test-service',
    models=[model],
    inference_config=inference_config,
    deployment_config=deployment_config,
    overwrite=True
)
test_service.wait_for_deployment(show_output=True)

Generating Docker build context.
Package creation Succeeded
Logging into Docker registry dc7c18e998184205acf38658cc51f079.azurecr.io
Logging into Docker registry dc7c18e998184205acf38658cc51f079.azurecr.io
Building Docker image from Dockerfile...
Step 1/5 : FROM dc7c18e998184205acf38658cc51f079.azurecr.io/azureml/azureml_43f212a02480d80122155ef1231227f6
 ---> 5a0df70074d4
Step 2/5 : COPY azureml-app /var/azureml-app
 ---> 73e29150c363
Step 3/5 : RUN mkdir -p '/var/azureml-app' && echo eyJhY2NvdW50Q29udGV4dCI6eyJzdWJzY3JpcHRpb25JZCI6ImQ3ZjM5MzQ5LWE2NmItNDQ2ZS1hYmE2LTAwNTNjMmNmMWMxMSIsInJlc291cmNlR3JvdXBOYW1lIjoiYW1sLXF1aWNrc3RhcnRzLTE2MjE4NyIsImFjY291bnROYW1lIjoicXVpY2stc3RhcnRzLXdzLTE2MjE4NyIsIndvcmtzcGFjZUlkIjoiZGM3YzE4ZTktOTgxOC00MjA1LWFjZjMtODY1OGNjNTFmMDc5In0sIm1vZGVscyI6e30sIm1vZGVsc0luZm8iOnt9fQ== | base64 --decode > /var/azureml-app/model_config_map.json
 ---> Running in 08f7165465b0
 ---> d28f43e539b6
Step 4/5 : RUN mv '/var/azureml-app/tmpuhsnrj0a.py' /var/azureml-app/main.py


TODO: In the cell below, send a request to the web service you deployed to test it.

In [42]:
r = requests.get(test_service.swagger_uri)
r.text

'{"swagger": "2.0", "info": {"title": "ML service", "description": "API specification for the Azure Machine Learning service ML service", "version": "1.0"}, "schemes": ["https"], "consumes": ["application/json"], "produces": ["application/json"], "securityDefinitions": {"Bearer": {"type": "apiKey", "name": "Authorization", "in": "header", "description": "For example: Bearer abc123"}}, "paths": {"/": {"get": {"operationId": "ServiceHealthCheck", "description": "Simple health check endpoint to ensure the service is up at any given point.", "responses": {"200": {"description": "If service is up and running, this response will be returned with the content \'Healthy\'", "schema": {"type": "string"}, "examples": {"application/json": "Healthy"}}, "default": {"description": "The service failed to execute due to an error.", "schema": {"$ref": "#/definitions/ErrorResponse"}}}}}, "/score": {"post": {"operationId": "RunMLService", "description": "Run web service\'s model and get the prediction out

In [44]:
# get some testdata to send a request
data = df.head(1).drop("HeartDisease", axis=1).to_dict(orient="records")
body = {"Inputs": [data],}
print(body)

{'Inputs': [[{'Age': 40, 'Sex': 1, 'RestingBP': 140, 'Cholesterol': 289, 'FastingBS': 0, 'MaxHR': 172, 'ExerciseAngina': 0, 'Oldpeak': 0.0, 'ST_Slope': 1, 'ChestPainType_ASY': 0, 'ChestPainType_ATA': 1, 'ChestPainType_NAP': 0, 'ChestPainType_TA': 0, 'RestingECG_LVH': 0, 'RestingECG_Normal': 1, 'RestingECG_ST': 0}]]}


In [45]:
import json, requests
# test against local deploymenz
uri = test_service.scoring_uri
requests.get("http://localhost:6789")
headers = {"Content-Type": "application/json"}
response = requests.post(uri, data=json.dumps(body), headers=headers)
print(response.json())

[0]


In [47]:
df.head(1)["HeartDisease"]

0    0
Name: HeartDisease, dtype: int64

In [41]:
test_service.update()

Generating Docker build context.
Package creation Succeeded
Logging into Docker registry dc7c18e998184205acf38658cc51f079.azurecr.io
Logging into Docker registry dc7c18e998184205acf38658cc51f079.azurecr.io
Building Docker image from Dockerfile...
Step 1/5 : FROM dc7c18e998184205acf38658cc51f079.azurecr.io/azureml/azureml_43f212a02480d80122155ef1231227f6
 ---> 5a0df70074d4
Step 2/5 : COPY azureml-app /var/azureml-app
 ---> 8b620a26d2b3
Step 3/5 : RUN mkdir -p '/var/azureml-app' && echo eyJhY2NvdW50Q29udGV4dCI6eyJzdWJzY3JpcHRpb25JZCI6ImQ3ZjM5MzQ5LWE2NmItNDQ2ZS1hYmE2LTAwNTNjMmNmMWMxMSIsInJlc291cmNlR3JvdXBOYW1lIjoiYW1sLXF1aWNrc3RhcnRzLTE2MjE4NyIsImFjY291bnROYW1lIjoicXVpY2stc3RhcnRzLXdzLTE2MjE4NyIsIndvcmtzcGFjZUlkIjoiZGM3YzE4ZTktOTgxOC00MjA1LWFjZjMtODY1OGNjNTFmMDc5In0sIm1vZGVscyI6e30sIm1vZGVsc0luZm8iOnt9fQ== | base64 --decode > /var/azureml-app/model_config_map.json
 ---> Running in e3394cae7f7b
 ---> dc8705077f36
Step 4/5 : RUN mv '/var/azureml-app/tmp9_l3m9h8.py' /var/azureml-app/main.py


In [33]:
# local deployment is working
test_service.delete()

Container (name:strange_booth, id:bf601216c7eeaca1d1eaa96aa16ddb1176a9ffd65e3215004ab78f49981aaf1f) cannot be killed.
Container has been successfully cleaned up.


In [48]:
service = Model.deploy(workspace=ws,
                       name=service_name,
                       models=[model],
                       inference_config=inference_config,
                       deployment_config=aci_config,
                       overwrite=True)
service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-10-27 13:07:36+00:00 Creating Container Registry if not exists.
2021-10-27 13:07:36+00:00 Registering the environment.
2021-10-27 13:07:39+00:00 Use the existing image.
2021-10-27 13:07:39+00:00 Generating deployment configuration.
2021-10-27 13:07:41+00:00 Submitting deployment to compute.
2021-10-27 13:07:44+00:00 Checking the status of deployment heart-failure-hyperdrive-service..
2021-10-27 13:10:24+00:00 Checking the status of inference endpoint heart-failure-hyperdrive-service.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [49]:
# send request to deployed web service
uri = service.scoring_uri
print(uri)
print(service.swagger_uri)
key, _ = service.get_keys()

http://5515eba9-2ad1-4140-a95c-9655f98c2592.southcentralus.azurecontainer.io/score
http://5515eba9-2ad1-4140-a95c-9655f98c2592.southcentralus.azurecontainer.io/swagger.json


In [50]:
headers = {"Content-Type": "application/json"}
headers["Authorization"] = f"Bearer {key}"
response = requests.post(uri, data=json.dumps(body), headers=headers)
print(response.json())

[0]


TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
print(service.get_logs())
service.delete()

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

